In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import time
import glob
%matplotlib inline
from sklearn import preprocessing
import tables
import h5py
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV

In [2]:
flights = pd.read_csv('BigData_Pruned_Drop.csv')
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,ANC,SEA,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,BOEING,737-990ER,0,9.1,13.5
1,1,4,DL,ANC,SEA,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,AIRBUS,A321-211,0,9.8,14.2
2,1,4,AA,LAX,PBI,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,BOEING,737-890,0,8.2,15.8
3,1,4,DL,DTW,PBI,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,BOEING,737-832,0,13.0,16.1
4,1,4,AA,LGA,PBI,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,AIRBUS,A320-232,0,12.5,15.5


In [3]:
orig_airports = flights['ORIGIN_AIRPORT'].tolist()
dest_airports = flights['DESTINATION_AIRPORT'].tolist()

In [4]:
le_airports = preprocessing.LabelEncoder()
le_airports.fit(orig_airports)
#print(le_airports.classes_)
enc_orig_airports = le_airports.transform(orig_airports)
enc_dest_airports = le_airports.transform(dest_airports) 

enc_dest_airports

array([277, 277, 235, ..., 174, 174, 174], dtype=int64)

In [5]:
flights['ORIGIN_AIRPORT'] = enc_orig_airports
flights['DESTINATION_AIRPORT'] = enc_dest_airports
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,BOEING,737-990ER,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,AIRBUS,A321-211,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,BOEING,737-890,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,BOEING,737-832,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,AIRBUS,A320-232,0,12.5,15.5


In [6]:
manufacts = flights['Manu'].tolist()

le_manu = preprocessing.LabelEncoder()
le_manu.fit(manufacts)
#print(le_airports.classes_)
enc_manufacts = le_manu.transform(manufacts)

flights['Manu'] = enc_manufacts
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,6,737-990ER,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,1,A321-211,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,6,737-890,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,6,737-832,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,1,A320-232,0,12.5,15.5


In [7]:
models = flights['Model'].tolist()

le_model = preprocessing.LabelEncoder()
le_model.fit(models)
#print(le_airports.classes_)
enc_models = le_model.transform(models)

flights['Model'] = enc_models
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,CLR,2012.0,6,53,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,CLR,2001.0,1,103,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,FEW,2009.0,6,46,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,FEW,2000.0,6,45,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,FEW,2014.0,1,102,0,12.5,15.5


In [8]:
arr_sky = flights['arr_sky'].tolist()
dept_sky = flights['dept_sky'].tolist()

le_sky = preprocessing.LabelEncoder()
le_sky.fit(arr_sky)
#print(le_airports.classes_)
enc_arr_sky = le_sky.transform(arr_sky)
enc_dept_sky = le_sky.transform(dept_sky)

flights['dept_sky'] = enc_dept_sky
flights['arr_sky'] = enc_arr_sky
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,AS,17,277,1448,-22,0,0,33.80,...,0,30.32,10,1,2012.0,6,53,0,9.1,13.5
1,1,4,DL,17,277,1448,-24,0,0,33.80,...,0,30.32,10,1,2001.0,1,103,0,9.8,14.2
2,1,4,AA,176,235,2330,-9,0,0,46.04,...,0,30.21,10,2,2009.0,6,46,0,8.2,15.8
3,1,4,DL,93,235,1086,-8,0,0,17.96,...,0,30.21,10,2,2000.0,6,45,0,13.0,16.1
4,1,4,AA,182,235,1035,-12,0,0,28.04,...,0,30.21,10,2,2014.0,1,102,0,12.5,15.5


In [9]:
airlines = flights['AIRLINE'].tolist()
le_airline = preprocessing.LabelEncoder()
le_airline.fit(airlines)
#print(le_airports.classes_)
enc_airlines = le_airline.transform(airlines)

flights['AIRLINE'] = enc_airlines
flights.head()

,MONTH,DOW,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DISTANCE,ARRIVAL_DELAY,DIVERTED,CANCELLED,dept_temp,...,arr_precip,arr_alti,arr_vsby,arr_sky,CertificateDate,Manu,Model,DAYS,DEPARTURE_HOUR,ARRIVAL_HOUR
0,1,4,1,17,277,1448,-22,0,0,33.80,...,0,30.32,10,1,2012.0,6,53,0,9.1,13.5
1,1,4,3,17,277,1448,-24,0,0,33.80,...,0,30.32,10,1,2001.0,1,103,0,9.8,14.2
2,1,4,0,176,235,2330,-9,0,0,46.04,...,0,30.21,10,2,2009.0,6,46,0,8.2,15.8
3,1,4,3,93,235,1086,-8,0,0,17.96,...,0,30.21,10,2,2000.0,6,45,0,13.0,16.1
4,1,4,0,182,235,1035,-12,0,0,28.04,...,0,30.21,10,2,2014.0,1,102,0,12.5,15.5


In [10]:
list(flights)

['MONTH',
 'DOW',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'DISTANCE',
 'ARRIVAL_DELAY',
 'DIVERTED',
 'CANCELLED',
 'dept_temp',
 'dept_dew',
 'dept_hum',
 'dept_windspd',
 'dept_precip',
 'dept_alti',
 'dept_vsby',
 'dept_sky',
 'arr_temp',
 'arr_dew',
 'arr_hum',
 'arr_windspd',
 'arr_precip',
 'arr_alti',
 'arr_vsby',
 'arr_sky',
 'CertificateDate',
 'Manu',
 'Model',
 'DAYS',
 'DEPARTURE_HOUR',
 'ARRIVAL_HOUR']

In [11]:
features = ['MONTH',
 'DOW',
 'AIRLINE',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'DISTANCE',
 'dept_temp',
 'dept_dew',
 'dept_hum',
 'dept_windspd',
 'dept_precip',
 'dept_alti',
 'dept_vsby',
 'dept_sky',
 'arr_temp',
 'arr_dew',
 'arr_hum',
 'arr_windspd',
 'arr_precip',
 'arr_alti',
 'arr_vsby',
 'arr_sky',
 'CertificateDate',
 'Manu',
 'Model',
 'DAYS',
 'DEPARTURE_HOUR',
 'ARRIVAL_HOUR']

In [12]:
features1 = [
 'DISTANCE',
 'dept_temp',
 'dept_dew',
 'dept_hum',
 'dept_windspd',
 'dept_alti',
 'dept_vsby',
 'dept_sky',
 'arr_temp',
 'arr_dew',
 'arr_hum',
 'arr_windspd',
 'arr_alti',
 'arr_vsby',
 'arr_sky',
 'CertificateDate',
 'DAYS',
 'DEPARTURE_HOUR',
 'ARRIVAL_HOUR']

In [13]:
X = flights[features]
y = flights["ARRIVAL_DELAY"]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=0)

In [16]:
parameters = {'criterion': ["mae"], 'min_samples_split' : [0.05, .15, .2], 'max_depth': [10,20,30,40], 'min_impurity_split' : [.00001, 0.0001, .001, .01]}

In [ ]:
regressor = DecisionTreeRegressor()
clf = GridSearchCV(regressor, parameters)
clf.fit(X_train, y_train)

In [16]:
X_new = SelectKBest(chi2, k=5).fit_transform(X, y)

ValueError: Input X must be non-negative.

In [38]:
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [39]:
regressor.feature_importances_

array([ 0.07744151,  0.05966789,  0.0461963 ,  0.06629816,  0.04822092,
        0.06490642,  0.02410822,  0.02040376,  0.05022941,  0.04289072,
        0.06589697,  0.04394302,  0.06109428,  0.01806178,  0.01936406,
        0.069282  ,  0.08175077,  0.06888609,  0.07135771])

In [40]:
clf = ExtraTreesClassifier()
clf = clf.fit(X_train, y_train)

KeyboardInterrupt: 

In [41]:
y_pred = regressor.predict(X_test)

In [ ]:
y_pred

In [42]:
mean_absolute_error(y_test, y_pred)

30.886544913909148

Mean_absolute_error if just using average: 25.339660117372873